<a href="https://colab.research.google.com/github/Garvit996/Data-Science-Garvit/blob/master/Text%20Clustering%20using%20Word%20Embedding%20%26%20Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up the Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
!pip install -q pyspark

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.functions as F

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec,StopWordsRemover
from pyspark.sql.functions import *
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pyspark.ml import Pipeline
from tensorflow.keras.layers import LeakyReLU,Dense,Input
from tensorflow.keras.models import Model,Sequential
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
sc=SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

Importing Data File

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Movies_Dataset_1000.csv to Movies_Dataset_1000.csv


In [ ]:
import io
import pandas as pd
# df = pd.read_csv(uploaded['Text_Data.csv'])
# import io
df = pd.read_csv(io.BytesIO(uploaded['Movies_Dataset_1000.csv']))
# df = pd.read_csv(io.StringIO(uploaded['Movies_Dataset.csv'].decode('cp1252')))
# Dataset is now stored in a Pandas Dataframe

Data Preprocessing

In [ ]:
df[["id","title","overview"]] = df[["id","title","overview"]].astype(str)
df = spark.createDataFrame(df)
df.show()    

+---+--------------------+--------------------+
| id|               title|            overview|
+---+--------------------+--------------------+
|  0|           Toy Story|Led by Woody, And...|
|  1|             Jumanji|When siblings Jud...|
|  2|    Grumpier Old Men|A family wedding ...|
|  3|   Waiting to Exhale|Cheated on, mistr...|
|  4|Father of the Bri...|Just when George ...|
|  5|                Heat|Obsessive master ...|
|  6|             Sabrina|An ugly duckling ...|
|  7|        Tom and Huck|A mischievous you...|
|  8|        Sudden Death|International act...|
|  9|           GoldenEye|James Bond must u...|
| 10|The American Pres...|Widowed U.S. pres...|
| 11|Dracula: Dead and...|When a lawyer sho...|
| 12|               Balto|An outcast half-w...|
| 13|               Nixon|An all-star cast ...|
| 14|    Cutthroat Island|Morgan Adams and ...|
| 15|              Casino|The life of the g...|
| 16|Sense and Sensibi...|Rich Mr. Dashwood...|
| 17|          Four Rooms|It's Ted the B

In [ ]:
documents = df.withColumn("tokenized", split(lower(col("overview")), " "))

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import Word2Vec,StopWordsRemover
from pyspark.sql.functions import *
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pyspark.ml import Pipeline
from tensorflow.keras.layers import LeakyReLU,Dense,Input
from tensorflow.keras.models import Model,Sequential
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
# Remove stop words
remover = StopWordsRemover(inputCol='tokenized', outputCol='text_splitted_v2')
df_words_no_stopw = remover.transform(documents).select("id","title","overview","tokenized",'text_splitted_v2')

Word Embedding

In [ ]:
# Word2Vec
word2Vec = Word2Vec(vectorSize=100, minCount=1, maxIter=100, inputCol="text_splitted_v2", outputCol="features")
model = word2Vec.fit(df_words_no_stopw)

In [ ]:
result = model.transform(df_words_no_stopw)

Dimensionality Reduction using **Autoencoders** 

In [ ]:
from pyspark.ml.functions import vector_to_array
#result2 = result.withColumn("col", vector_to_array("features")).select(["id"] + [col(("col"))[i] for i in range(100)])
result3 = result.withColumn("col",vector_to_array("features")).select([col(("col"))[i] for i in range(100)])
result_pd = result3.toPandas()
result_pd

In [ ]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    self.encoder = Sequential(
        [
          Dense(50, activation="relu"),
          Dense(20, activation="relu"),
          Dense(3, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(20, activation="relu"),
          Dense(50, activation="relu"),
          Dense(100, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
    
auto_encoder = AutoEncoders(len(result_pd.columns))

auto_encoder.compile(
    loss='mse',
    optimizer='adam'
)

history = auto_encoder.fit(result_pd,result_pd,epochs= 12,batch_size=32)

In [ ]:
auto_encoder.summary()

In [ ]:
encoder_layer = auto_encoder.get_layer('sequential_2')
reduced_df = pd.DataFrame(encoder_layer.predict(result_pd))
reduced_df = reduced_df.add_prefix('feature_')
reduced_df

Clustering - K-means & Elbow Plot

In [ ]:



X = reduced_df
#print(X)
data = X

sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(data)
    data["clusters"] = kmeans.labels_
    #print(data["clusters"])
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

In [ ]:
kmeans9 = KMeans(n_clusters=9)
y_kmeans9 = kmeans9.fit_predict(reduced_df)
y_kmeans9 = pd.DataFrame(y_kmeans9, columns=['Cluster No'])

In [ ]:
Auto = pd.concat([reduced_df,y_kmeans9],axis=1)
#Auto.rename(columns={'Cluster Number': 'Cluster No',}, inplace=True)
Auto

Vizualisation

In [ ]:

df = Auto
#feature_0	feature_1	feature_2	clusters	Column1
#fig = px.scatter_3d(TF123a_pandas, x='col1', y='col2', z='col3',color='prediction')
fig = px.scatter_3d(df, x='feature_0', y='feature_1', z='feature_2', size='clusters', color='clusters')
                   # hover_data=  )
fig.update_layout(scene_zaxis_type="log")
fig.show()